<a href="https://colab.research.google.com/github/krvamsi96/pro-machine-learning-algorithms/blob/master/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#datasets:https://www.hackerearth.com/challenges/competitive/hackerearth-deep-learning-challenge-auto-tag-images-gala/
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train=pd.read_csv("/content/gdrive/My Drive/Dddd/train.csv")
test=pd.read_csv("/content/gdrive/My Drive/Dddd/test.csv")

In [0]:
train.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [0]:
test.head()

,Image
0,image6245.jpg
1,image10409.jpg
2,image8692.jpg
3,image10517.jpg
4,image2580.jpg


In [0]:
Class_map={'Food':0,'Attire':1,'Decorationandsignage':2,'misc':3}
inverse_map={0:'Food',1:'Attire',2:'Decorationandsignage',3:'misc'}
train['Class']=train['Class'].map(Class_map)

In [0]:
train['Class']


0       0
1       3
2       1
3       0
4       1
       ..
5978    0
5979    1
5980    0
5981    0
5982    1
Name: Class, Length: 5983, dtype: int64

In [0]:
train_img=[]
train_label=[]
j=0
path="/content/gdrive/My Drive/Dddd/Train Images"
for i in tqdm(train['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(150,150))
    img=img.astype('float32')
    train_img.append(img)
    train_label.append(train['Class'][j])
    j=j+1

100%|██████████| 5983/5983 [29:21<00:00,  3.40it/s]


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
data=ImageDataGenerator(featurewise_center=False,
                        samplewise_center=False,
                        featurewise_std_normalization=False,
                        samplewise_std_normalization=False,
                        zca_whitening=False,
                         rotation_range=10,
                        zoom_range = 0.1,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        horizontal_flip=True,
                        vertical_flip=False)
data.fit(train_img)

In [0]:
test_img=[]
path="/content/gdrive/My Drive/Dddd/Test Images"
for i in tqdm(test['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(150,150))
    img=img.astype('float32')
    test_img.append(img)

100%|██████████| 3219/3219 [17:16<00:00,  3.11it/s]


In [0]:
train_img=np.array(train_img)
test_img=np.array(test_img)
train_label=np.array(train_label)
print(train_img.shape)
print(test_img.shape)
print(train_label.shape)

(5983, 150, 150, 3)
(3219, 150, 150, 3)
(5983,)


In [0]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [0]:
base_model=VGG16(include_top=False, weights='imagenet',input_shape=(150,150,3), pooling='avg')

model=Sequential()
model.add(base_model)

model.add(Dense(256,activation='relu'))
model.add(Dense(4,activation='softmax'))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
58892288/58889256 [==============================] - 2s 0us/step


In [0]:
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

base_model.trainable=False

reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)


In [0]:
callbacks = [reduce_learning_rate]
model.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(data.flow(train_img, to_categorical(train_label,4), batch_size=32),
                    epochs=20,callbacks=callbacks)

Epoch 1/20
187/187 [==============================] - 33s 175ms/step - loss: 1.6340 - acc: 0.6370
Epoch 2/20
187/187 [==============================] - 28s 149ms/step - loss: 0.7660 - acc: 0.7299
Epoch 3/20
187/187 [==============================] - 28s 149ms/step - loss: 0.6095 - acc: 0.7623
Epoch 4/20
187/187 [==============================] - 28s 149ms/step - loss: 0.5423 - acc: 0.7921
Epoch 5/20
187/187 [==============================] - 28s 150ms/step - loss: 0.4741 - acc: 0.8202
Epoch 6/20
187/187 [==============================] - 28s 149ms/step - loss: 0.4286 - acc: 0.8367
Epoch 7/20
187/187 [==============================] - 28s 147ms/step - loss: 0.3908 - acc: 0.8533
Epoch 8/20
187/187 [==============================] - 27s 147ms/step - loss: 0.3831 - acc: 0.8593
Epoch 9/20
187/187 [==============================] - 28s 148ms/step - loss: 0.3492 - acc: 0.8663
Epoch 10/20
187/187 [==============================] - 28s 149ms/step - loss: 0.3256 - acc: 0.8785
Epoch 11/20
187/187

In [0]:
labels = model.predict(test_img)
print(labels[:4])
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
print(class_label[:3])
submission = pd.DataFrame({ 'Image': test.Image, 'Class': class_label })
submission.head(10)

[[9.9399346e-01 5.7384989e-04 3.3458245e-03 2.0868562e-03]
 [3.1292313e-01 4.8855104e-06 3.9057380e-05 6.8703288e-01]
 [9.8741502e-01 1.0260729e-09 5.1396812e-12 1.2585021e-02]
 [9.9796766e-01 3.5682478e-11 4.1040445e-11 2.0323223e-03]]
['Food', 'misc', 'Food']


,Image,Class
0,image6245.jpg,Food
1,image10409.jpg,misc
2,image8692.jpg,Food
3,image10517.jpg,Food
4,image2580.jpg,Attire
5,image848.jpg,Attire
6,image1232.jpg,Attire
7,image8386.jpg,misc
8,image9828.jpg,misc
9,image9033.jpg,Attire


In [0]:
test.head